##Loading Stuff

###Installing

In [25]:
!pip install fitz
!pip install pymupdf
!pip install path
!pip install binarytree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Importing

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import nltk
from nltk.corpus import stopwords
nltk.download('words')
nltk.download("popular")
nltk.download('stopwords')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading 

True

In [44]:
import fitz
from path import Path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from collections import OrderedDict
from binarytree import Node

Use Textract (https://textract.readthedocs.io/en/stable/) to extract text from the PDF files. Use NLTK to tokenize them and the Porter's Stemmer to stem the tokens. Use arrays instead of lists since we have a fixed corpus. If time permits, maybe implement wildcards and a ranking system?

##Path and Postings List Initialization

In [29]:
# create an empty dictionary for the postings list
postings = {}
# initialize the PorterStemmer and stopwords
ps = nltk.wordnet.WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [30]:
# pth = Path('/content/drive/Othercomputers/My Laptop (1)/Academics/3-2/CS F469 Information Retrieval/Assignment-1_2/SamplePolicyDocs')

In [31]:
# doc = fitz.open('/content/drive/Othercomputers/My Laptop (1)/Academics/3-2/CS F469 Information Retrieval/Assignment-1_2/SamplePolicyDocs/Auto/7thEditionPolicy.pdf')

##Summarizer

In [65]:
import heapq
import re
from typing import List

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

def summarize(text: str, summary_length: int = 5) -> str:
    # Tokenize the text into sentences and words
    sentences = sent_tokenize(text)
    words = [word.lower() for word in word_tokenize(text)]
    # Remove stop words and punctuation
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word.isalpha() and word not in stop_words]
    # Create a frequency table for the words
    word_freq = dict()
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1
    # Compute the sentence scores based on word frequencies
    sentence_scores = dict()
    for sentence in sentences:
        sentence_words = [word.lower() for word in word_tokenize(sentence)]
        sentence_score = sum([word_freq.get(word, 0) for word in sentence_words])
        sentence_scores[sentence] = sentence_score
    # Select the top N sentences with the highest scores
    summary_sentences = heapq.nlargest(summary_length, sentence_scores, key=sentence_scores.get)
    # Return the summary as a single string
    summary = ' '.join(summary_sentences)
    return summary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Spelling Corrector

In [32]:
from nltk.corpus import words
correct_words = words.words()

In [33]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(correct_words)

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def edits3(word):
    "All edits that are three edits away from `word`."
    return (e3 for e2 in edits2(word) for e3 in edits1(e2))

def known_edits2(word):
    "The subset of `words` that appear in the dictionary of WORDS and are two edits away from `word`."
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in WORDS)

def known_edits3(word):
    "The subset of `words` that appear in the dictionary of WORDS and are three edits away from `word`."
    return set(e3 for e2 in edits2(word) for e3 in edits1(e2) if e3 in WORDS)

def candidates_v2(word):
    "Generate possible spelling corrections for word with edits up to 3."
    return (known([word]) or known(edits1(word)) or known_edits2(word) or known_edits3(word) or [word])

def correction_v2(word): 
    "Most probable spelling correction for word with edits up to 3."
    return max(candidates_v2(word), key=P)

##Acronym Expansion using Beautiful Soup

In [34]:
import requests
from bs4 import BeautifulSoup

def acronym_expander(acronym):
    url = f'https://www.acronymfinder.com/{acronym}.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    full_forms_text = soup.find_all('a', {'class': 'r5'})
    titles = []
    for i in full_forms_text:
      titles.append(i['title'])
    if len(titles) == 0:
        return None
    for i in range(0,len(titles)):
      titles[i] = titles[i].split('- ')[1]
    return titles
# Example usage
print(acronym_expander('defn'))

['Deficiency', 'Definition']


##Creating the Postings List

In [35]:
pdf_dir=Path('SamplePolicyDocs/Auto/')
# loop through each PDF file in the directory
for pdf_path in pdf_dir.glob("*.pdf"):
    # open the PDF file with fitz
    doc = fitz.open(str(pdf_path))
    for page_num, page in enumerate(doc):
        # extract the text blocks from the page
        blocks = page.get_text("blocks")
        # loop through each text block in the page
        for block_num, block in enumerate(blocks):
            # tokenize the text block
            tokens = word_tokenize(block[4])
            # stem the tokens
            stemmed_tokens = [ps.lemmatize(token) for token in tokens]
            # loop through each stemmed token in the text block
            for position, token in enumerate(stemmed_tokens):
                # create the document ID tuple
                doc_id = (pdf_path.stem.split('.')[0], page_num, block_num)
                # add the term and document ID to the postings list
                if token not in postings:
                    # create a tuple with the document IDs, frequency of the term, and positions
                    postings[token] = ([doc_id], 1, [position])
                else:
                    # if the document ID is already in the postings list
                    if doc_id in postings[token][0]:
                        # increment the frequency of the term and add the new position
                        freq = postings[token][1]
                        positions = postings[token][2]
                        positions.append(position)
                        postings[token] = (postings[token][0], freq+1, positions)
                    # if the document ID is not in the postings list
                    else:
                        # add the document ID, frequency of the term, and position
                        postings[token] = (postings[token][0]+[doc_id], postings[token][1]+1, postings[token][2]+[position])

# sort the dictionary for easier searching
postings = OrderedDict(sorted(postings.items()))

In [36]:
# for term, posting in postings.items():
#     print(term + ":")
#     print("\tDocument IDs:", posting[0])
#     print("\tFrequency:", posting[1])

In [47]:
def boolean_retrieval(query, postings):
    # tokenize the query string
    words = query.split()
    final=""
    for word in words:
      if(correction_v2(word)!=word and word!="AND" and word!="OR" and word!="NOT"):
        print("Did you mean: "+word+":"+correction_v2(word))
        input3=input("yes or no \n")
        if(input3=="yes"):
          word=correction_v2(word)
      final=final+word+" "
    tokens = word_tokenize(final)
    for i in range(0,len(tokens)):
      if tokens[i] != "NOT" and tokens[i] != "OR" and tokens[i] != "AND":
        tokens[i] = ps.lemmatize(tokens[i])
      else:
        continue
    # initialize result set
    result = set()
    # iterate over tokens in query
    for token in tokens:
        # check if token is a NOT operand
        if token.startswith('NOT'):
            # remove the NOT operand from the token
            term = token[3:]
            # get the document IDs that contain the term
            docs = set(postings.get(term, ([], 0, []))[0])
            # remove the document IDs from the result set
            result.difference_update(docs)
        # check if token is an AND operand
        elif token == 'AND':
            # skip to next token
            continue
        # check if token is an OR operand
        elif token == 'OR':
            # skip to next token
            continue
        # token is a term
        else:
            # get the document IDs that contain the term
            docs = set(postings.get(token, ([], 0, []))[0])
            # add the document IDs to the result set
            result.update(docs)
    # iterate over tokens in query again
    for i, token in enumerate(tokens):
        # check if token is an AND operand
        if token == 'AND':
            # get the document IDs that match the next term in the query
            next_docs = set(postings.get(tokens[i+1], ([], 0, []))[0])
            # intersect the result set with the next set of document IDs
            result.intersection_update(next_docs)
        # check if token is an OR operand
        elif token == 'OR':
            # get the document IDs that match the next term in the query
            next_docs = set(postings.get(tokens[i+1], ([], 0, []))[0])
            # union the result set with the next set of document IDs
            result.update(next_docs)
    # return the result set
    return result

In [74]:
def print_query_res(query):
  result = boolean_retrieval(query, postings)
  f.write("\nRetrieved Documents: {}".format(len(result)))
  print("\nRetrieved Documents: {}".format(len(result)))
  for r in result:
    if r[0] == '1215E':
      doc = fitz.open('SamplePolicyDocs/Auto/'+r[0]+'.2.pdf')
    else:
      doc = fitz.open('SamplePolicyDocs/Auto/'+r[0]+'.pdf')
    pg = doc[r[1]]
    bl = pg.get_text("blocks")
    print("\nFile Path: {}".format(str(doc)))
    print("Page Number: {}".format(int(str(pg).split(' ')[1]) + 1))
    print(bl[r[2]][4])
    print("A short summary: "+ summarize(bl[r[2]][4]))
    f.write("\nFile Path: {}".format(str(doc)))
    f.write("Page Number: {}".format(int(str(pg).split(' ')[1]) + 1))
    f.write(bl[r[2]][4])
    f.write("A short summary: "+ summarize(bl[r[2]][4]))

In [39]:
# print_query_res("paid AND your AND premium")

##Binary Tree Implementation

In [49]:
class Node:
    def __init__(self, char: str):
        self.char = char
        self.children = []
        self.postings = []
        
class BinaryTree:
    def __init__(self, postings):
        self.root = Node('')
        self.build_tree(postings)
        
    def build_tree(self, postings):
        for term, (doc_ids, freq, pos) in postings.items():
            current_node = self.root
            for char in term:
                child_node = None
                for child in current_node.children:
                    if child.char == char:
                        child_node = child
                        break
                if child_node is None:
                    child_node = Node(char)
                    current_node.children.append(child_node)
                current_node = child_node
            current_node.postings.append((term, doc_ids, freq, pos))
            
    def search(self, query: str):
        results = []
        query_node = self.root
        for char in query:
            if char == '*':
                results += self.traverse(query_node)
                break
            else:
                child_node = None
                for child in query_node.children:
                    if child.char == char:
                        child_node = child
                        break
                if child_node is None:
                    return []
                query_node = child_node
        results += query_node.postings
        return results
    
    def traverse(self, node: Node):
        results = []
        if len(node.postings) > 0:
            results += node.postings
        for child in node.children:
            results += self.traverse(child)
        return results

##Reverse Binary Tree Implementation

In [50]:
class ReverseNode:
    def __init__(self, char: str):
        self.char = char
        self.children = []
        self.postings = []
        
class ReverseBinaryTree:
    def __init__(self, postings):
        self.root = ReverseNode('')
        self.build_tree(postings)
        
    def build_tree(self, postings):
        for term, (doc_ids, freq, pos) in postings.items():
            current_node = self.root
            for char in reversed(term):
                child_node = None
                for child in current_node.children:
                    if child.char == char:
                        child_node = child
                        break
                if child_node is None:
                    child_node = ReverseNode(char)
                    current_node.children.append(child_node)
                current_node = child_node
            current_node.postings.append((term, doc_ids, freq, pos))
            
    def search(self, query: str):
        results = []
        query_node = self.root
        for char in reversed(query):
            if char == '*':
                results += self.traverse(query_node)
                break
            else:
                child_node = None
                for child in query_node.children:
                    if child.char == char:
                        child_node = child
                        break
                if child_node is None:
                    return []
                query_node = child_node
        results += query_node.postings
        return results
    
    def traverse(self, node: ReverseNode):
        results = []
        if len(node.postings) > 0:
            results += node.postings
        for child in node.children:
            results += self.traverse(child)
        return results


In [51]:
# tree = ReverseBinaryTree(postings)
# results1 = tree.search("*at")
# result = []
# for i in range(0, len(results1)):
#   #print(results1[i])
#   for j in results1[i][1]:
#     result.append(j)
# print(set(result))

##Search

Now, make the posting list by linking every distinct term with the document index or ptr in the tokens array(in this case, the block of text) as discussed in class. Clean up the token array to remove empty arrays and stop words.

In [75]:
def intersection(list1, list2):
    intersection_list = []
    for element in list1:
        if element in list2 and element not in intersection_list:
            intersection_list.append(element)
    return intersection_list
from collections import defaultdict

def phrase_query(query, postings):
    # Tokenize the query and stem the tokens
    tokens = word_tokenize(query)
    query_tokens = [ps.lemmatize(token) for token in tokens]
    #print(query_tokens)
    # Create a dictionary to store the positions of each query term
    term_positions = defaultdict(list)
    for i, token in enumerate(query_tokens):
        if token in postings:
            term_positions[token].extend(postings[token][2])
    try:
    # Find the positions where the terms occur together in each document
      candidate_docs = set(postings[query_tokens[0]][0])
      for token in query_tokens[1:]:
          if token in postings:
              candidate_docs = candidate_docs.intersection(postings[token][0])
    except KeyError:
      return -1
    
    candidate_docs = sorted(candidate_docs)
    #print(candidate_docs)
    return candidate_docs

def print_phrase_query_res(query):
  if phrase_query(query, postings) == -1:
    print("Not Found.")
    return
  else:
    result = phrase_query(query, postings)
    print("\nRetrieved Documents: {}".format(len(result)))
    f.write("\nRetrieved Documents: {}".format(len(result)))
    for r in result:
      if r[0] == '1215E':
        doc = fitz.open('SamplePolicyDocs/Auto/'+r[0]+'.2.pdf')
      else:
        doc = fitz.open('SamplePolicyDocs/Auto/'+r[0]+'.pdf')
      pg = doc[r[1]]
      bl = pg.get_text("blocks")
      print("\nFile Path: {}".format(str(doc)))
      print("Page Number: {}".format(int(str(pg).split(' ')[1]) + 1))
      print(bl[r[2]][4])
      print("A short summary: "+ summarize(bl[r[2]][4]))
      f.write("\nFile Path: {}".format(str(doc)))
      f.write("Page Number: {}".format(int(str(pg).split(' ')[1]) + 1))
      f.write(bl[r[2]][4])
      f.write("A short summary: "+ summarize(bl[r[2]][4]))
        
def print_wildcard_query_res(result):
    print("\nRetrieved Documents: {}".format(len(result)))
    f.write("\nRetrieved Documents: {}".format(len(result)))
    for r in result:
      if r[0] == '1215E':
        doc = fitz.open('SamplePolicyDocs/Auto/'+r[0]+'.2.pdf')
      else:
        doc = fitz.open('SamplePolicyDocs/Auto/'+r[0]+'.pdf')
      pg = doc[r[1]]
      bl = pg.get_text("blocks")
      print("\nFile Path: {}".format(str(doc)))
      print("Page Number: {}".format(int(str(pg).split(' ')[1]) + 1))
      print(bl[r[2]][4])
      print("A short summary: "+ summarize(bl[r[2]][4]))
      f.write("\nFile Path: {}".format(str(doc)))
      f.write("Page Number: {}".format(int(str(pg).split(' ')[1]) + 1))
      f.write(bl[r[2]][4])
      f.write("A short summary: "+ summarize(bl[r[2]][4]))

user_input=input("Enter a number (1-3)"+"\n"+"1. Wildcard"+"\n"+"2. Phrase Query"+"\n"+"3. Boolean Query"+"\n")
f = open("results.txt", "w")
if user_input == "1":
    input1=input("Enter query:")
    s1=""
    s2="*"
    flag= False
    for char in input1:
        if char == '*':
            flag= True
        elif flag:
            s2+=char
        else:
            s1+=char
    s1+='*'
    
    if(s1!="*"):
        tree = BinaryTree(postings)
        results1 = tree.search(s1)
    if(s2!="*"):
        rev_tree = ReverseBinaryTree(postings)
        results2 = rev_tree.search(s2)
    if(s1=="*" and s2!="*"):
        results=results2
        result = []
        for i in range(0, len(results)):
          #print(results1[i])
          for j in results[i][1]:
            result.append(j)
        print_wildcard_query_res(set(result))
        # for term, doc_ids, freq, pos in results:
        #   print(f"{term}: {doc_ids}, {freq},{pos}")
    elif(s2=="*" and s1!="*"):
        results=results1
        result = []
        for i in range(0, len(results)):
        #print(results1[i])
          for j in results[i][1]:
            result.append(j)
        print_wildcard_query_res(set(result))
        # for term, doc_ids, freq, pos in results:
        #   print(f"{term}: {doc_ids}, {freq},{pos}")
    elif(s1!="*" and s2!="*"):
        results=intersection(results1,results2)
        result = []
        for i in range(0, len(results)):
        #print(results1[i])
          for j in results[i][1]:
            result.append(j)
        print_wildcard_query_res(set(result))
        # for term, doc_ids, freq, pos in results:
        #   print(f"{term}: {doc_ids}, {freq},{pos}")
    else:
        print("Invalid input")
    
    
elif user_input == "2":
    input1=input("Enter query:")
    words = input1.split()
    final=""
    final1=""
    for word in words:
      if(acronym_expander(word)==None):
        if(correction_v2(word)!=word):
          print("Did you mean: "+word+":"+correction_v2(word))
          input3=input("yes or no \n")
          if(input3=="yes"):
            word=correction_v2(word)
        final1=final1+word+" "
        final=final+word+" "
      else:
        print("For "+word+" did you mean:")
        l=acronym_expander(word)
        k=len(l)
        if(k>5):
          k=5
        for i in range(0,k):
          print(i+1)
          print(l[i])
        input2=input("Enter the index of the expansion if it matches else enter 0.\n")
        if(input2=="0"):
          input2=word
        else:
          input2=l[int(input2)-1]
        final=final+input2+" "
        final1=final1+word+" "
    ##print(final1, final)
    print_phrase_query_res(final1)
    print_phrase_query_res(final)
elif user_input =="3":
    input1=input("Enter query:")
    print_query_res(input1)
else:
    print("Invalid input")

Enter a number (1-3)
1. Wildcard
2. Phrase Query
3. Boolean Query
2
Enter query:paid your premium
For paid did you mean:
1
Preserve the American Dream
Enter the index of the expansion if it matches else enter 0.
0
For premium did you mean:
1
Home Box Office (premium movie channel on USA cable systems)
2
Fast Track (instant picture of premium/loss information in the casualty industry)
3
Cinemax (premium cable channel)
4
Premium Processing (US Department of Homeland Security)
5
Premium Only Plan (employee benefit plan)
Enter the index of the expansion if it matches else enter 0.
0

Retrieved Documents: 2

File Path: Document('/content/drive/Othercomputers/My Laptop (1)/Academics/3-2/CS F469 Information Retrieval/Assignment-1_2/SamplePolicyDocs/Auto/7thEditionPolicy.pdf')
Page Number: 9
When you purchased this Part you were given the choice of either
excluding yourself, or yourself and household members, from some
or all of the PIP coverage. The portion of each claim you may have
agreed n